**Import libraries**

In [1]:
import os
from IPython.display import clear_output
import time
#Astrick import fucntions
from func_cif_v03 import *
from func_hkl import *
import matplotlib.pyplot as plt

**File management**

In [2]:
# Here enters the mode of calculation
# We have throughly compute "1", & data check "2"
cal_mode = "draw_xrd"

# Here enters the folder of your CIF files. (NO slash on left)
in_dir = "CIFs"

# Folder for your output files.
out_dir = "database_xrds/ExampleXRDs/"
if not os.path.exists(out_dir+"/"):
        os.makedirs(out_dir+"/")

# Calculate hkl planes based on wanted accurracy
hkl_max = 10

# Use script func_hkl to generate hkl matrix
print("Generating hkl matrix")
hkl_info = hkl(hkl_max)
print("hkl_info done!\n") # This print is usually ignored

# Create log file
logFolder = f"archive_log/"
if not os.path.exists(logFolder):
        os.makedirs(logFolder)
logFile = logFolder + "logfile.txt"
with open(logFile, "a") as cal_log:
    pass


Generating hkl matrix
Generating hkl_info
hkl_info done!



In [3]:
# Find current working directory
cwdir = os.getcwd()
# Count availble CIF files and create directories
cif_count = 0
for path, dirs, files in os.walk(in_dir):
    for file in files:
        if file.endswith('.cif'):
            cif_count += 1

# Calculate XRD and show progress
x_min = 0
x_max = 90
x_step = 0.01
cif_cal_count = 0
cif_fail_count = 0

with open(logFile, "r") as cal_log:
    cal_log_lines = cal_log.readlines()
        
for path, dirs, files in os.walk(in_dir):
    for file in files:
        if file.endswith('.cif'):
            # Write a log file
            print("Calculating " + path + ": " + file)
            print("Progress: ", str(cif_cal_count), "/", str(cif_count))
            # Here combines cwd and subdirectory
            full_dir = "{}/{}".format(cwdir, path)
            fileDir = "{}/{}.txt".format(out_dir, re.split(r"[.]", file)[0])
            # Here record timing
            time_start = time.time()
            # Check existing XRD output files in output folder to avoid repeated calculations.
            if False and cal_mode == "draw_xrd" and os.path.isfile(fileDir):
                cif_return = "Cancel"
            else:
                # Here calls main function to calculate XRD and output
                cif_return = cif(full_dir, file, out_dir, hkl_info, x_min, x_max, x_step, cal_mode)
            # Here record timing
            time_cost = format(time.time() - time_start, '.3f')
            cif_cal_count += 1
            clear_output(wait = True)
            print("Time cost:" + time_cost)
            # LOG
            if "Cancel" in cif_return:
                print("Result of ", file, ": ", cif_return, "\n", "Progress: ", str(cif_cal_count), "/", str(cif_count))
            elif "GOOD" in cif_return:
                with open(logFile, "a") as cal_log:
                    print(file+" "+cif_return+" "+str(time_cost), file=cal_log)
                print("Result of ", file, ": ", cif_return, ", ", "Good! logged", "\n", 
                    'Time cost: ', time_cost)
            elif "Fail" in cif_return:
                error_judge = False
                # Examine log file
                for line in cal_log_lines:
                    if file in line:
                        error_judge = True
                        print("Result of ", file, ": ", cif_return, ", ", "Cancelled: log exist", "\n", "Progress: ", str(cif_cal_count), "/", str(cif_count))
                # If not found, log this new fail
                if error_judge == False:
                    with open(logFile, "a") as cal_log:
                        print(file+" "+cif_return, file=cal_log)
                    print("Result of ", file, ": ", cif_return, ", ", "Failed! logged", "\n", "Progress: ", str(cif_cal_count), "/", str(cif_count))
                cif_fail_count += 1
print("Pipeline complete! Total Fail: ", cif_fail_count, "/", cif_count)

Time cost:1.638
Result of  icsd_000005.cif :  GOOD! ,  Good! logged 
 Time cost:  1.638
Pipeline complete! Total Fail:  0 / 11
